**Лабораторная работа №6.**Применение сверточных нейронных сетей (многоклассовая классификация)


1.   Загрузите данные. Разделите исходный набор данных на обучающую и валидационную выборки.
2.   Реализуйте глубокую нейронную сеть со сверточными слоями. Какое качество классификации получено? Какая архитектура сети была использована?
3.   Примените дополнение данных (data augmentation). Как это повлияло на качество классификатора? 
4.   Поэкспериментируйте с готовыми нейронными сетями (например, AlexNet, VGG16, Inception и т.п.), применив передаточное обучение. Как это повлияло на качество классификатора? Можно ли было обойтись без него?
Какой максимальный результат удалось получить на контрольной выборке?



In [0]:
# Install Kaggle library
!pip install -q kaggle

import os
os.environ['KAGGLE_USERNAME'] = "awful1996" # username from the json file
os.environ['KAGGLE_KEY'] = "5e55f76a1cc7a4772bd7803cba8fb2c1" # key from the json file
#!kaggle datasets download -d iarunava/happy-house-dataset # api copied from kaggle
!kaggle datasets download -d datamunge/sign-language-mnist

 66% 41.0M/62.6M [00:00<00:00, 26.1MB/s]
100% 62.6M/62.6M [00:00<00:00, 67.8MB/s]


In [0]:
!ls

american_sign_language.PNG  sample_data		     sign_mnist_test.csv
amer_sign2.png		    sign-language-mnist.zip  sign_mnist_train
amer_sign3.png		    sign_mnist_test	     sign_mnist_train.csv


In [0]:
!unzip sign-language-mnist.zip

Archive:  sign-language-mnist.zip
  inflating: amer_sign2.png          
  inflating: amer_sign3.png          
  inflating: american_sign_language.PNG  
  inflating: sign_mnist_test.csv     
  inflating: sign_mnist_test/sign_mnist_test.csv  
  inflating: sign_mnist_train.csv    
  inflating: sign_mnist_train/sign_mnist_train.csv  


In [0]:
!ls

american_sign_language.PNG  sample_data		     sign_mnist_test.csv
amer_sign2.png		    sign-language-mnist.zip  sign_mnist_train
amer_sign3.png		    sign_mnist_test	     sign_mnist_train.csv


In [0]:
import pandas as pd
df = pd.read_csv('sign_mnist_train.csv')
print(df)

testDF = pd.read_csv('sign_mnist_test.csv')
print(testDF)


       label  pixel1  pixel2  pixel3  ...  pixel781  pixel782  pixel783  pixel784
0          3     107     118     127  ...       206       204       203       202
1          6     155     157     156  ...       175       103       135       149
2          2     187     188     188  ...       198       195       194       195
3          2     211     211     212  ...       225       222       229       163
4         13     164     167     170  ...       157       163       164       179
...      ...     ...     ...     ...  ...       ...       ...       ...       ...
27450     13     189     189     190  ...       234       200       222       225
27451     23     151     154     157  ...       195       195       195       194
27452     18     174     174     174  ...       203       202       200       200
27453     17     177     181     184  ...        47        64        87        93
27454     23     179     180     180  ...       197       205       209       215

[27455 rows x 7

In [0]:
def getData(df):
  y = df["label"].values.reshape((df.shape[0], 1)) 
  x = df.drop(columns="label").values.reshape((df.shape[0], 28, 28, 1))
  print(y.shape, x.shape)
  return x, y

x, y = getData(df)
tX, tY = getData(testDF)

(27455, 1) (27455, 28, 28, 1)
(7172, 1) (7172, 28, 28, 1)


In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.25)

**Задание 2.**
Реализуйте глубокую нейронную сеть со сверточными слоями. Какое качество классификации получено? Какая архитектура сети была использована?

Была получена точность 93 процента


In [0]:
from tensorflow.keras import layers, models, Sequential

model = Sequential([
    layers.Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)),
    layers.Conv2D(32, kernel_size=5, activation='relu'),
    layers.Dropout(0.25),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.Dropout(0.25),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Dropout(0.25),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(500, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(25, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [0]:
import tensorflow as tf

model.fit(x_train, y_train, validation_split=0.1, epochs=100, callbacks=[
      tf.keras.callbacks.EarlyStopping(
        patience=2,
        restore_best_weights=True,
        monitor='accuracy'
      )
    ])

Epoch 1/100
580/580 [==============================] - 5s 9ms/step - loss: 3.0719 - accuracy: 0.1263 - val_loss: 1.9795 - val_accuracy: 0.4971
Epoch 2/100
580/580 [==============================] - 5s 8ms/step - loss: 1.1424 - accuracy: 0.6333 - val_loss: 0.3806 - val_accuracy: 0.9383
Epoch 3/100
580/580 [==============================] - 5s 8ms/step - loss: 0.3862 - accuracy: 0.8723 - val_loss: 0.1172 - val_accuracy: 0.9883
Epoch 4/100
580/580 [==============================] - 5s 8ms/step - loss: 0.2489 - accuracy: 0.9207 - val_loss: 0.0915 - val_accuracy: 0.9883
Epoch 5/100
580/580 [==============================] - 5s 8ms/step - loss: 0.1934 - accuracy: 0.9377 - val_loss: 0.0396 - val_accuracy: 0.9971
Epoch 6/100
580/580 [==============================] - 5s 8ms/step - loss: 0.1519 - accuracy: 0.9513 - val_loss: 0.0398 - val_accuracy: 0.9971
Epoch 7/100
580/580 [==============================] - 5s 8ms/step - loss: 0.1315 - accuracy: 0.9608 - val_loss: 0.0433 - val_accuracy: 0.9917

In [0]:
model.evaluate(tX,  tY)

225/225 [==============================] - 1s 4ms/step - loss: 0.2069 - accuracy: 0.9334


[0.20685824751853943, 0.9333519339561462]

**Задание 3.**
Примените дополнение данных (data augmentation). Как это повлияло на качество классификатора? 

После применение качество упало до 5 процентов


In [0]:
model = Sequential([
    layers.Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)),
    layers.Conv2D(32, kernel_size=5, activation='relu'),
    layers.Dropout(0.25),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.Dropout(0.25),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Dropout(0.25),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(500, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(25, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

train_generator = datagen.flow(x_train, y_train, batch_size=32)
val_generator = datagen.flow(x_val, y_val, batch_size=32)
test_generator = datagen.flow(tX, tY, batch_size=32)

In [0]:
EPOCHS = 100
BS = 32

model.fit_generator(
  train_generator,
  epochs=100,
  validation_data=(x_val,y_val), steps_per_epoch=len(x_train) // 32)

Epoch 1/100
643/643 [==============================] - 13s 20ms/step - loss: 3.2701 - accuracy: 0.0173 - val_loss: 2.9611 - val_accuracy: 1.4569e-04
Epoch 2/100
643/643 [==============================] - 12s 19ms/step - loss: 2.8287 - accuracy: 0.0168 - val_loss: 2.4577 - val_accuracy: 0.0935
Epoch 3/100
643/643 [==============================] - 12s 19ms/step - loss: 2.4743 - accuracy: 0.0523 - val_loss: 1.8554 - val_accuracy: 0.0622
Epoch 4/100
643/643 [==============================] - 12s 19ms/step - loss: 2.0677 - accuracy: 0.0498 - val_loss: 1.4050 - val_accuracy: 0.0172
Epoch 5/100
643/643 [==============================] - 12s 19ms/step - loss: 1.6249 - accuracy: 0.0453 - val_loss: 0.8157 - val_accuracy: 0.0609
Epoch 6/100
643/643 [==============================] - 12s 19ms/step - loss: 1.3345 - accuracy: 0.0450 - val_loss: 0.5770 - val_accuracy: 0.0402
Epoch 7/100
643/643 [==============================] - 12s 19ms/step - loss: 1.1157 - accuracy: 0.0407 - val_loss: 0.4387 - va

KeyboardInterrupt: ignored

In [0]:
model.evaluate(tX,  tY)

225/225 [==============================] - 1s 5ms/step - loss: 0.0989 - accuracy: 0.0506


[0.09888710826635361, 0.05061349645256996]

**Задание 4.**
Поэкспериментируйте с готовыми нейронными сетями (например, AlexNet, VGG16, Inception и т.п.), применив передаточное обучение. Как это повлияло на качество классификатора? Можно ли было обойтись без него?
Какой максимальный результат удалось получить на контрольной выборке?

При обучении точность была около 4-х процентов, поэтому дообучивать не было смысла, скорее всего это из-за того что ImageNet не содержит фотографий жестов и не обучался изначально на этих данных, поэтому не стоит использовать передаточное обучение на этом наборе данных


In [0]:
from keras.applications import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model

base_model=MobileNet(weights='imagenet',include_top=False) 

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x)
preds=Dense(25,activation='softmax')(x) 
model=Model(inputs=base_model.input,outputs=preds)

In [0]:
print(len(model.layers))
#for layer in model.layers:
#    layer.trainable=False

# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

92


In [0]:
import numpy as np
def transform(dataset):
  newDataset = list()
  for x in dataset:
    x = np.repeat(x, 3, 2)
    newDataset.append(x)
  return np.array(newDataset)

newTrainX = transform(x_train)
newValX = transform(x_val)

print(newTrainX.shape, newValX.shape)

(20591, 28, 28, 3) (6864, 28, 28, 3)


In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(newTrainX, y_train, validation_data=(newValX, y_val), epochs=15)

Train on 20591 samples, validate on 6864 samples
Epoch 1/15
20591/20591 [==============================] - 37s 2ms/step - loss: 3.2080 - accuracy: 0.0454 - val_loss: 3.1988 - val_accuracy: 0.0457
Epoch 2/15
20591/20591 [==============================] - 32s 2ms/step - loss: 3.1947 - accuracy: 0.0476 - val_loss: 3.1898 - val_accuracy: 0.0457
Epoch 3/15
20591/20591 [==============================] - 32s 2ms/step - loss: 3.1882 - accuracy: 0.0476 - val_loss: 3.1848 - val_accuracy: 0.0457
Epoch 4/15
20591/20591 [==============================] - 32s 2ms/step - loss: 3.1845 - accuracy: 0.0476 - val_loss: 3.1819 - val_accuracy: 0.0457
Epoch 5/15
20591/20591 [==============================] - 32s 2ms/step - loss: 3.1821 - accuracy: 0.0476 - val_loss: 3.1800 - val_accuracy: 0.0457
Epoch 6/15
20591/20591 [==============================] - 32s 2ms/step - loss: 3.1805 - accuracy: 0.0476 - val_loss: 3.1786 - val_accuracy: 0.0457
Epoch 7/15
20591/20591 [==============================] - 32s 2ms/ste